In [1]:
!pip install pandas
!pip install scikit-learn

In [2]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

In [3]:
data_path_idle = '/Users/vika/Desktop/data/idle'
data_path_running = '/Users/vika/Desktop/data/running'
data_path_stairs = '/Users/vika/Desktop/data/stairs'
data_path_walking = '/Users/vika/Desktop/data/walking'

def load_data(data_path, label):
    data = []
    print(f"Loading data from {data_path}...")
    try:
        for filename in os.listdir(data_path):
            if filename.endswith(".csv"):
                file_path = os.path.join(data_path, filename)
                df = pd.read_csv(file_path)
                
#(time domain features)
                df['mean'] = df.mean(axis=1)
                df['std'] = df.std(axis=1)
                df['min'] = df.min(axis=1)
                df['max'] = df.max(axis=1)
                
                df['label'] = label
                data.append(df)
        print(f"Successfully loaded {len(data)} files from {data_path}.")
        return pd.concat(data, ignore_index=True)
    except FileNotFoundError:
        print(f"Error: Directory not found - {data_path}")
        return None

# Об'єднання даних з усіх класів
df_idle = load_data(data_path_idle, 'idle')
df_running = load_data(data_path_running, 'running')
df_stairs = load_data(data_path_stairs, 'stairs')
df_walking = load_data(data_path_walking, 'walking')

if all(df is not None for df in [df_idle, df_running, df_stairs, df_walking]):
    min_samples = min(len(df_idle), len(df_running), len(df_stairs), len(df_walking))
    df = pd.concat([df_idle.sample(min_samples), df_running.sample(min_samples), df_stairs.sample(min_samples), df_walking.sample(min_samples)], ignore_index=True)

    X = df[['mean', 'std', 'min', 'max']]
    y = df['label']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    svm_model = make_pipeline(StandardScaler(), SVC())
    rf_model = RandomForestClassifier()

    svm_model.fit(X_train, y_train)
    rf_model.fit(X_train, y_train)

    y_pred_svm = svm_model.predict(X_test)
    y_pred_rf = rf_model.predict(X_test)

    print("SVM Classification Report:")
    print(classification_report(y_test, y_pred_svm))

    print("\nRandom Forest Classification Report:")
    print(classification_report(y_test, y_pred_rf))

Loading data from /Users/vika/Desktop/data/idle...
Successfully loaded 1039 files from /Users/vika/Desktop/data/idle.
Loading data from /Users/vika/Desktop/data/running...
Successfully loaded 3408 files from /Users/vika/Desktop/data/running.
Loading data from /Users/vika/Desktop/data/stairs...
Successfully loaded 165 files from /Users/vika/Desktop/data/stairs.
Loading data from /Users/vika/Desktop/data/walking...
Successfully loaded 1850 files from /Users/vika/Desktop/data/walking.
SVM Classification Report:
              precision    recall  f1-score   support

        idle       0.91      0.98      0.95       985
     running       0.83      0.64      0.72      1020
      stairs       0.54      0.76      0.63       998
     walking       0.66      0.49      0.56       957

    accuracy                           0.72      3960
   macro avg       0.73      0.72      0.71      3960
weighted avg       0.73      0.72      0.71      3960


Random Forest Classification Report:
             